In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd

def optimal_portfolio(returns):
    
    print(1)
    n = len(returns)
    returns = np.asmatrix(returns)
    
    print(2)
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    print(3)
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    print(4)
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    print(5)
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    
    print(6)
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    
    print(7)
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    print(8)
    
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    print(9)
    return np.asarray(wt), returns, risks

In [4]:
import numpy
from pymongo import MongoClient
from datetime import datetime, timedelta as td

def GetDataSerieFromMongo(stock,startDate='',endDate=''):
   
    client = MongoClient()
    db = client.Project
    
    if startDate=='' and endDate=='':
        data = list(db.HistPrices.find({'BBGTicker':stock})) 
    else:
        d1 = datetime.strptime(startDate, '%Y-%m-%d')
        d2 = datetime.strptime(endDate, '%Y-%m-%d') 
        delta = (d2-d1).days+1
        dateList=[]
        
        for i in range(delta):
            dateList.append((d1+td(days=i)).strftime('%Y-%m-%d'))
            
        data = list(db.HistPrices.find({'Date':{"$in":dateList},'BBGTicker':stock})) 
    
    headers= ['BBGTicker','Close','Date','High','Low','Name','Open','Volume','_id']    
    records=[]
    records.append(headers)
    for i in range(len(data)):
        records.append([data[i]['BBGTicker'],
                        data[i]['Close'],
                        data[i]['Date'],
                        data[i]['High'],
                        data[i]['Low'],
                        data[i]['Name'],
                        data[i]['Open'],
                        data[i]['Volume'],
                        data[i]['_id']])
        
    return numpy.asarray(records)

In [5]:
from pymongo import MongoClient
client = MongoClient()
db = client.Project

startDate = ''
endDate = ''
stocks =[x['BBGTicker'] for x in list(db.Stocks.find({}))]

AllPrices = [] 
if startDate=='' and endDate=='':
    print("suck1")
else:
    print("suck2")
    #d1 = datetime.strptime(startDate, '%Y-%m-%d')
    #d2 = datetime.strptime(endDate, '%Y-%m-%d') 
    #delta = (d2-d1).days+1
    #for i in range(delta):
        #db.HistSignals.delete_many({'Date':(d1+td(days=i)).strftime('%Y-%m-%d')})
        
        
for s in stocks:
    print(s)
    if startDate=='' and endDate=='':
        data = GetDataSerieFromMongo(s) 
    else:
        d = (datetime.strptime(startDate, '%Y-%m-%d') - td(days=90)).strftime('%Y-%m-%d')
        data = GetDataSerieFromMongo(s,d,endDate)
            
    AllPrices.append(data)
print("Get LOST.")

suck1
EXPD
ESRX
FB
FAST
FISV
GRMN
GILD
HSIC
ILMN
INCY
INTC
INTU
ISRG
KLAC
GMCR
KHC
LRCX
LBTYA
LMCA
LVNTA
LLTC
MAR
MAT
MU
MSFT
MDLZ
MNST
MYL
NTAP
NFLX
NVDA
ORLY
PCAR
PAYX
QCOM
REGN
ROST
SNDK
SBAC
STX
SIRI
SWKS
SPLS
SBUX
SRCL
SYMC
TSLA
TXN
PCLN
TSCO
TRIP
VRSK
VRTX
VIAB
VIP
VOD
WBA
WDC
WFM
WYNN
XLNX
YHOO
FOXA
FOX
ATVI
ADBE
AKAM
ALXN
GOOGL
GOOG
AMZN
AAL
AMGN
ADI
AAPL
AMAT
ADSK
ADP
AVGO
BIDU
BBBY
BIIB
BMRN
BRCM
CHRW
CA
CELG
CERN
CHTR
CHKP
CSCO
CTXS
CTSH
CMCSK
CMCSA
COST
DISCA
DISCK
DISH
DLTR
EBAY
EA
Get LOST.


In [6]:
#AllPrices[1]
#numpy.array(AllPrices[1:,1], dtype='f8')
#AllPrices[1][1:,1]
#AllPrices[1][1:,1].pct_change()

# Preallocate DataFrame
AllRets =pd.DataFrame()

for stockIndex in range(0, len(stocks)-1):
    #print(stock)
    #stockIndex = 1
    Prices = pd.DataFrame({'Close':AllPrices[stockIndex][1:,1]},index=AllPrices[stockIndex][1:,2])
    Rets = pd.concat([Prices["Close"].pct_change()], axis=1).dropna()
    stockName = AllPrices[stockIndex][1,0]
    Rets.columns=[stockName]
    AllRets = pd.concat([AllRets, Rets])
    print(stockName)
AllRets.head()
#AllRets = Rets

EXPD
ESRX
FB
FAST
FISV
GRMN
GILD
HSIC
ILMN
INCY
INTC
INTU
ISRG
KLAC
GMCR
KHC
LRCX
LBTYA
LMCA
LVNTA
LLTC
MAR
MAT
MU
MSFT
MDLZ
MNST
MYL
NTAP
NFLX
NVDA
ORLY
PCAR
PAYX
QCOM
REGN
ROST
SNDK
SBAC
STX
SIRI
SWKS
SPLS
SBUX
SRCL
SYMC
TSLA
TXN
PCLN
TSCO
TRIP
VRSK
VRTX
VIAB
VIP
VOD
WBA
WDC
WFM
WYNN
XLNX
YHOO
FOXA
FOX
ATVI
ADBE
AKAM
ALXN
GOOGL
GOOG
AMZN
AAL
AMGN
ADI
AAPL
AMAT
ADSK
ADP
AVGO
BIDU
BBBY
BIIB
BMRN
BRCM
CHRW
CA
CELG
CERN
CHTR
CHKP
CSCO
CTXS
CTSH
CMCSK
CMCSA
COST
DISCA
DISCK
DISH
DLTR
EBAY


,AAL,AAPL,ADBE,ADI,ADP,ADSK,AKAM,ALXN,AMAT,AMGN,...,VIP,VOD,VRSK,VRTX,WBA,WDC,WFM,WYNN,XLNX,YHOO
1990-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
AllRets.tail()

,AAL,AAPL,ADBE,ADI,ADP,ADSK,AKAM,ALXN,AMAT,AMGN,...,VIP,VOD,VRSK,VRTX,WBA,WDC,WFM,WYNN,XLNX,YHOO
2016-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
AllRets[stocks[1:10]].head()

,ESRX,FB,FAST,FISV,GRMN,GILD,HSIC,ILMN,INCY
1990-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#import stocks.py
#start = '10/17/2010'
#end = '10/17/2016'

#r_avg = map(lambda x: x.mean(), AllRets)

# covariance of asset returns
#sigma = numpy.array(stocks.cov_matrix(symbols, start, end, 'y'))

#AllRets[stocks[0]].mean()
#symbols = stocks # For simplicity
#lambda s: stocks.avg_return(s, start, end, 'y')
# average yearly return for each stock
#r_avg = map(lambda s: stocks.avg_return(s, start, end, 'y'), symbols)

In [ ]:
weights, returns, risks = optimal_portfolio(AllRets[stocks[1:10]])

plt.plot(stds, means, 'o')
plt.ylabel('mean')
plt.xlabel('std')
plt.plot(risks, returns, 'y-o')